<a href="https://colab.research.google.com/github/CanKeles5/Predict-NG/blob/main/TEIA%C5%9E_LSTM_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forecasting NG production.

In [ ]:
import torch
torch.cuda.is_available()
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)

In [ ]:
device

device(type='cpu')

In [ ]:
%matplotlib inline

In [ ]:
! pip install pickle5

     |████████████████████████████████| 133kB 5.6MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219263 sha256=e45523a04cbca7cc0d3f3580a0a26e965cacb0da2449b710632a1dd69ef39976
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
import pickle5
from math import sqrt
from datetime import datetime
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from array import array
import numpy as np

In [ ]:
with open('/content/20210101-epias-raw.pkl', 'rb') as f:
    data = pickle5.load(f)

In [ ]:
new_data = data['naturalGas']

In [ ]:
new_data

0        2326.01
1        2334.78
2        2157.76
3        2084.72
4        2034.68
          ...   
67263    8912.15
67264    9068.42
67265    9164.25
67266    9240.85
67267    9189.95
Name: naturalGas, Length: 67268, dtype: float64

In [ ]:
data = new_data.to_numpy()

In [ ]:
data

array([2326.01, 2334.78, 2157.76, ..., 9164.25, 9240.85, 9189.95])

In [ ]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out):
	# flatten data
  print("train.shape: " + str(train.shape))
  #data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
  data = np.expand_dims(train, axis=1)
  print(len(data))

  X, y = list(), list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(data)):
		# define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
		# ensure we have enough data for this instance
    if out_end <= len(data):
      x_input = data[in_start:in_end]
      x_input = x_input.reshape((len(x_input), 1))
      X.append(x_input)
      y.append(data[in_end:out_end])
		# move along one time step
    in_start += 1

  return np.array(X), np.array(y)

In [ ]:
train_X, train_y = to_supervised(data[0:42000], 3, 1)
val_X, val_y = to_supervised(data[42000:54000], 3, 1)
test_X, test_y = to_supervised(data[54000:60068], 3, 1)

train.shape: (42000,)
42000
train.shape: (12000,)
12000
train.shape: (6068,)
6068


In [ ]:
train_Y.shape

(41997, 1, 1)

In [ ]:
train_X[0]

array([[2326.01],
       [2334.78],
       [2157.76]])

In [ ]:
train_X[1]

array([[2334.78],
       [2157.76],
       [2084.72]])

In [ ]:
train_y[0]

array([[2084.72]])

In [ ]:
class LSTM(nn.Module):

  def __init__(self):
    super(LSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=3, hidden_size=100, num_layers=2)
    self.FC = nn.Linear(100, 1)

  def forward(self, input):
    pred, _ = self.lstm(input)

    pred = self.FC(pred.view(100))

    return pred

In [ ]:
model = LSTM().to(device)

In [ ]:
sum(p.numel() for p in model.parameters())

122901

In [ ]:
loss_function = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
train_loss_hist = []
test_loss_hist = []

In [ ]:
for epoch in range(50):
  train_loss = 0.0
  test_loss = 0.0

  for i in range(train_X.shape[0]):
    data = torch.Tensor(train_X[i]).unsqueeze(0).unsqueeze(0).squeeze(3)
    data = data.to(device)

    optimizer.zero_grad()

    prediction = model(data)

    #print("Train --- prediction: " + str(prediction) + " , GT: " + str(train_y[i]))

    loss = loss_function(prediction, torch.Tensor([train_y[i]]).to(device))

    train_loss += loss.item()

    loss.backward()
    optimizer.step()

  with torch.no_grad():
    for i in range(test_X.shape[0]):
      data = torch.Tensor(test_X[i]).unsqueeze(0).unsqueeze(0).squeeze(3)
      data = data.to(device)

      prediction = model(data)

      #print("Validate --- prediction: " + str(prediction) + " , GT: " + str(test_y[i]))

      loss = loss_function(prediction, torch.Tensor([test_y[i]]).to(device))
      test_loss += loss.item()

  train_loss = train_loss / train_X.shape[0]
  test_loss = test_loss / test_X.shape[0]

  train_loss_hist.append(train_loss)
  test_loss_hist.append(test_loss)

  print("Train loss: " +  str(train_loss) + " --- Test loss: " + str(test_loss))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Train loss: 11661.988880067922 --- Test loss: 7469.457218733093
Train loss: 11627.925441135818 --- Test loss: 7436.965918723513
Train loss: 11595.473437894068 --- Test loss: 7404.519937201718
Train loss: 11563.03070765427 --- Test loss: 7372.074350974389
Train loss: 11530.588253250593 --- Test loss: 7339.628613341631
Train loss: 11498.145844735733 --- Test loss: 7307.18305929739
Train loss: 11465.703417690656 --- Test loss: 7274.73768060972
Train loss: 11433.260981272382 --- Test loss: 7242.292101748876
Train loss: 11400.818541702942 --- Test loss: 7209.846380761155
Train loss: 11368.376081912345 --- Test loss: 7177.400805201141
Train loss: 11335.933530699087 --- Test loss: 7144.955336822484
Train loss: 11303.49092650716 --- Test loss: 7112.509410192076
Train loss: 11271.048253226496 --- Test loss: 7080.063819964504
Train loss: 11238.605658100392 --- Test loss: 7047.6181952291745
Train loss: 11206.163062493239 --- Test loss: 7015.172256572878
Train loss: 11173.720464437965 --- Test los